<a href="https://colab.research.google.com/github/Yixuan-042/CASA0025/blob/main/notebooks/W04_week4_final_postgis2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [1]:
%pip install duckdb duckdb-engine jupysql
%pip install duckdb leafmap lonboard
import duckdb
import leafmap
import pandas as pd
%load_ext sql

In [2]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [3]:
%sql duckdb:///:memory:
# %sql duckdb:///path/to/file.db

In [4]:
%%sql

SELECT * FROM duckdb_extensions();

,extension_name,loaded,installed,install_path,description,aliases,extension_version,install_mode,installed_from
0,arrow,False,False,,A zero-copy data integration between Apache Ar...,[],,None,
1,autocomplete,False,False,,Adds support for autocomplete in the shell,[],,None,
2,aws,False,False,,Provides features that depend on the AWS SDK,[],,None,
3,azure,False,False,,Adds a filesystem abstraction for Azure blob s...,[],,None,
4,delta,False,False,,Adds support for Delta Lake,[],,None,
5,excel,False,False,,Adds support for Excel-like format strings,[],,None,
6,fts,False,True,(BUILT-IN),Adds support for Full-Text Search Indexes,[],,STATICALLY_LINKED,
7,httpfs,False,True,/root/.duckdb/extensions/v1.1.3/linux_amd64_gc...,Adds support for reading and writing files ove...,"[http, https, s3]",v1.1.3,REPOSITORY,core
8,iceberg,False,False,,Adds support for Apache Iceberg,[],,None,
9,icu,True,True,(BUILT-IN),Adds support for time zones and collations usi...,[],,STATICALLY_LINKED,


In [5]:
%%sql

INSTALL httpfs;
LOAD httpfs;

,Success


In [6]:
url= "https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip"
leafmap.download_file(url, unzip=True)

nyc_data.zip already exists. Skip downloading. Set overwrite=True to overwrite.


'/content/nyc_data.zip'

In [8]:
%%sql

CREATE TABLE nyc_census_blocks AS SELECT * FROM '/content/nyc_census_blocks.shp';

,Success


In [10]:
%%sql

CREATE TABLE nyc_homicides AS SELECT * FROM '/content/nyc_homicides.shp';

,Success


In [11]:
%%sql

CREATE TABLE nyc_neighborhoods AS SELECT * FROM '/content/nyc_neighborhoods.shp';

,Success


In [12]:
%%sql

CREATE TABLE nyc_streets AS SELECT * FROM '/content/nyc_streets.shp';

,Success


In [13]:
%%sql

CREATE TABLE nyc_subway_stations AS SELECT * FROM '/content/nyc_subway_stations.shp';

,Success


In [14]:
%%sql

FROM nyc_census_blocks;

,BLKID,POPN_TOTAL,POPN_WHITE,POPN_BLACK,POPN_NATIV,POPN_ASIAN,POPN_OTHER,BORONAME,geom
0,360850009001000,97,51,32,1,5,8,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 55, 3, 13, 73, 151, 8..."
1,360850020011000,66,52,2,0,7,5,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 178, 58, 13, 73, 72, ..."
2,360850040001000,62,14,18,2,25,3,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 82, 227, 12, 73, 55, ..."
3,360850074001000,137,92,12,0,13,20,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 204, 85, 13, 73, 103,..."
4,360850096011000,289,230,0,0,32,27,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 107, 247, 12, 73, 7, ..."
...,...,...,...,...,...,...,...,...,...
38789,360050295001004,328,267,14,2,8,37,The Bronx,"[2, 4, 0, 0, 0, 0, 0, 0, 118, 130, 16, 73, 15,..."
38790,360050295002002,0,0,0,0,0,0,The Bronx,"[2, 4, 0, 0, 0, 0, 0, 0, 244, 138, 16, 73, 135..."
38791,360050419004001,0,0,0,0,0,0,The Bronx,"[2, 4, 0, 0, 0, 0, 0, 0, 78, 3, 17, 73, 85, 28..."
38792,360050255002001,480,96,96,20,12,256,The Bronx,"[2, 4, 0, 0, 0, 0, 0, 0, 4, 123, 16, 73, 154, ..."


In [15]:
%%sql

FROM nyc_homicides;

,INCIDENT_D,BORONAME,NUM_VICTIM,PRIMARY_MO,ID,WEAPON,LIGHT_DARK,YEAR,geom
0,2008-01-01,Brooklyn,1,None,7,gun,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,2008-01-04,Manhattan,1,None,14,gun,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,2008-01-05,Queens,1,None,15,gun,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,2008-01-04,Queens,1,None,16,knife,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,2008-01-05,Queens,1,None,18,gun,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...
3977,2010-10-11,The Bronx,1,None,4269,gun,None,2010,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3978,2010-10-06,The Bronx,1,None,4271,knife,None,2010,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3979,2011-07-26,The Bronx,1,None,4282,gun,None,2011,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3980,2011-07-28,The Bronx,1,None,4284,gun,None,2011,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [16]:
%%sql

FROM nyc_neighborhoods;

,BORONAME,NAME,geom
0,Brooklyn,Bensonhurst,"[2, 4, 0, 0, 0, 0, 0, 0, 54, 71, 14, 73, 198, ..."
1,Manhattan,East Village,"[2, 4, 0, 0, 0, 0, 0, 0, 35, 215, 14, 73, 139,..."
2,Manhattan,West Village,"[2, 4, 0, 0, 0, 0, 0, 0, 161, 95, 14, 73, 212,..."
3,The Bronx,Throggs Neck,"[2, 4, 0, 0, 0, 0, 0, 0, 128, 232, 17, 73, 174..."
4,The Bronx,Wakefield-Williamsbridge,"[2, 4, 0, 0, 0, 0, 0, 0, 83, 85, 17, 73, 17, 2..."
...,...,...,...
124,Brooklyn,Red Hook,"[5, 4, 0, 0, 0, 0, 0, 0, 18, 0, 14, 73, 149, 8..."
125,Queens,Douglastown-Little Neck,"[2, 4, 0, 0, 0, 0, 0, 0, 251, 165, 19, 73, 76,..."
126,Queens,Whitestone,"[5, 4, 0, 0, 0, 0, 0, 0, 17, 75, 18, 73, 86, 2..."
127,Queens,Steinway,"[5, 4, 0, 0, 0, 0, 0, 0, 124, 87, 16, 73, 87, ..."


In [17]:
%%sql

FROM nyc_streets;

,ID,NAME,ONEWAY,TYPE,geom
0,1,Shore Pky S,None,residential,"[1, 4, 0, 0, 0, 0, 0, 0, 23, 66, 15, 73, 202, ..."
1,2,None,None,footway,"[1, 4, 0, 0, 0, 0, 0, 0, 80, 57, 15, 73, 35, 1..."
2,3,Avenue O,None,residential,"[1, 4, 0, 0, 0, 0, 0, 0, 228, 63, 15, 73, 219,..."
3,4,Walsh Ct,None,residential,"[1, 4, 0, 0, 0, 0, 0, 0, 139, 62, 15, 73, 102,..."
4,5,None,None,motorway_link,"[1, 4, 0, 0, 0, 0, 0, 0, 176, 53, 15, 73, 16, ..."
...,...,...,...,...,...
19086,17378,FDR Dr,yes,motorway,"[1, 4, 0, 0, 0, 0, 0, 0, 191, 114, 14, 73, 206..."
19087,17382,FDR Dr,yes,motorway,"[1, 4, 0, 0, 0, 0, 0, 0, 18, 115, 14, 73, 192,..."
19088,19039,FDR Dr,None,motorway,"[1, 4, 0, 0, 0, 0, 0, 0, 98, 50, 15, 73, 233, ..."
19089,19053,FDR Dr,None,motorway,"[1, 4, 0, 0, 0, 0, 0, 0, 59, 51, 15, 73, 94, 1..."


In [18]:
%%sql

FROM nyc_subway_stations;

,OBJECTID,ID,NAME,ALT_NAME,CROSS_ST,LONG_NAME,LABEL,BOROUGH,NGHBHD,ROUTES,TRANSFERS,COLOR,EXPRESS,CLOSED,geom
0,1.0,376.0,Cortlandt St,None,Church St,"Cortlandt St (R,W) Manhattan","Cortlandt St (R,W)",Manhattan,None,"R,W","R,W",YELLOW,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,2.0,2.0,Rector St,None,None,Rector St (1) Manhattan,Rector St (1),Manhattan,None,1,1,RED,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,3.0,1.0,South Ferry,None,None,South Ferry (1) Manhattan,South Ferry (1),Manhattan,None,1,1,RED,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,4.0,125.0,138th St,Grand Concourse,Grand Concourse,"138th St / Grand Concourse (4,5) Bronx","138th St / Grand Concourse (4,5)",Bronx,None,"4,5","4,5",GREEN,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,5.0,126.0,149th St,Grand Concourse,Grand Concourse,149th St / Grand Concourse (4) Bronx,149th St / Grand Concourse (4),Bronx,None,4,"2,4,5",GREEN,express,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,487.0,909.0,JFK Terminal 8,None,None,"JFK Terminal 8, Queens",JFK Terminal 8,Queens,None,None,None,AIR-BLUE,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
487,488.0,903.0,Federal Circle,Rental Car,None,"Federal Circle / Rental Car, Queens",Federal Circle / Rental Car,Queens,None,None,None,AIR-BLUE,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
488,489.0,902.0,Long Term Parking,None,None,"Long Term Parking, Queens",Long Term Parking,Queens,None,None,None,AIR-BLUE,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
489,490.0,901.0,Howard Beach,None,159th Ave,"Howard Beach, Queens",Howard Beach,Queens,None,None,A,AIR-BLUE,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

1. **What subway station is in \'Little Italy\'? What subway route is it on?**

In [19]:
%%sql

SELECT s.name, s.routes
FROM nyc_subway_stations AS s
JOIN nyc_neighborhoods AS n
ON ST_Contains(n.geom, s.geom)
WHERE n.name = 'Little Italy';

,NAME,ROUTES
0,Spring St,6


2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

4. **What neighborhood has the highest population density (persons/km2)?**


When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [ ]:
%pip install duckdb duckdb-engine jupysql

In [ ]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [ ]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

You can set a spatial index on each of these tables as follows:

`CREATE INDEX index_name ON table_name USING RTREE(geom);`

## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.